# Container for data

In [1]:
import numpy as np
class Program:
    # this class contains in a structured manner matrices and biases
    # which encode the semantics of the program network, in the same fashion
    # of the source code of a program
    def __init__(self, topology , activation_functions_list):
        self.topology = topology
        self.activation_functions_list = activation_functions_list
        self.W = [ 0 for t in topology]
        self.b = [ 0 for t in topology]
    def random_init_weights(self):
        for layer in range(len(self.topology[:-1])):
            self.W[layer] = np.random.normal( size = (self.topology[layer + 1] , self.topology[layer] ) )
            #for i in range(5):
            #    r = np.random.choice(len(self.W[layer]))
            #    c = np.random.choice(len(self.W[layer].T))
            #    self.W[layer][r,c] = 0.   # adding some sparsity
                
            self.b[layer] = np.random.normal( size = self.topology[layer + 1] )
    def print(self):
        for i,layer in enumerate(self.topology[:-1]):
            print("layer %d->%d "%(i,i+1))
            print("W.shape = %s \t b.shape = %s" % (str(self.W[i].shape),str(self.b[i].shape)))
            

In [2]:
rete = Program([5,10,4,1], ["RELU","RELU","LINEAR"])
rete.random_init_weights()
rete.print()

layer 0->1 
W.shape = (10, 5) 	 b.shape = (10,)
layer 1->2 
W.shape = (4, 10) 	 b.shape = (4,)
layer 2->3 
W.shape = (1, 4) 	 b.shape = (1,)


# Intermediate representation

In [24]:
class c:
    # every element is a node in the tree.
    # the first argument denotes the name of the node ; the possible next arguments are a list of the sons.
    # Note that nodes without sons are simply leaf (eg. ADD, MUL,...)
    def __init__(self, ID, *args):
        self.id   = ID
        self.sons = list()
        for arg in args:
            self.sons.append(arg)
    def print(self,level = 0):
        print( ("\t" * level) + str(self.id) )
        for s in self.sons:
            s.print(level + 1)
    def __str__(self):
        ret = str(self.id)
        if len(self.sons) > 0:
            ret += "("
            for s in self.sons:
                ret += str(s)
                if s != self.sons[-1]:
                    ret += ','
            ret += ")"
        return ret
C = c("BINOP", c("ADD"), c("CONST",c(1)), c("CONST",c(2)) )        
str( C )

'BINOP(ADD,CONST(1),CONST(2))'

In [37]:
def IR(program, compile_time_data = True):
    # for every layer
    print("y = input")
    ret = list()
    for layer,t in enumerate(program.topology[1:]):
        f = program.activation_functions_list[layer]
        R_offset = t
        for i in range(len(program.b[layer])): # for every row
            #print("R[%d] = 0" % (i))
            #print("MOVE( TEMP(%d) , CONST(0) )" % (i + R_offset))
            IR_instruction = c(
                    "MOVE",
                     c("TEMP" , c(i + R_offset) ),
                     c("CONST", c(0))
            )
            print(str(IR_instruction))
            ret.append(IR_instruction)
        for i in range(len(program.W[layer])): # for every row
            for j in range(len(program.W[layer].T)): # for every column
                #print(" >> R[%d] +=  W[%d->%d][%d,%d] * Y[%d]" % (i,layer,layer+1,i,j,j))
                if compile_time_data:
                    if( program.W[layer][i,j] != 0):
                        # if there is zero is useless to compute the contribution
                        #print("->MOVE(TEMP(%d),BINOP(ADD,TEMP(%d),BINOP(MUL,CONST(%f),TEMP(%d)))) " % (i + R_offset,i + R_offset, program.W[layer][i,j],i )) # COMPILE DATA HYPOTHESIS
                        IR_instruction = c(
                            "MOVE",
                            c("TEMP",
                                c(i + R_offset)
                            ),
                            c("BINOP",
                                c("ADD"),
                                c("TEMP",
                                    c(i + R_offset)
                                ),
                                c("BINOP",
                                    c("MUL"),
                                    c("CONST",
                                         c(program.W[layer][i,j])
                                     ),
                                    c("TEMP",
                                     c(i)
                                     )
                                )
                            )
                        )
                        print(IR_instruction)
                        ret.append(IR_instruction)
                    else:
                        print("# here there was a 0 so we exploit sparsity ")
                else:
                    print("#TODO")
                    #N = len(program.W[layer])
                    #ijth_element_position = i*N + j
                    #print("MOVE(TEMP(%d),BINOP(ADD,TEMP(%d),BINOP(MUL,MEM(BINOP(ADD,&W,CONST(%d))),TEMP(%d))))" % (i + R_offset,i + R_offset, ijth_element_position,i )) # COMPILE DATA HYPOTHESIS
        for i in range(len(program.b[layer])): # for every row
            # a priori in compile time since the amount of "repetitions" doens't scale quadratically , in opposite to weights
            #print("->MOVE(TEMP(%d),BINOP(ADD,TEMP(%d),CONST(%f)))" % (i, i + R_offset , program.b[layer][i]) )
            IR_instruction = c("MOVE",
                                c("TEMP",
                                 i
                                 ),
                                c("BINOP",
                                 c("ADD"),
                                 c("TEMP",
                                   c(i + R_offset)
                                  ),
                                 c("CONST",
                                  c(program.b[layer][i])
                                  )
                                 )
                                )
            print((IR_instruction))
            ret.append(IR_instruction)
        for i in range(len(program.b[layer])): # for every row
            #print("y[%d] = f(y[%d])" % (i,i))
            #print("MOVE( TEMP(%d) , CALL( %s, TEMP(%d) ) )" % (i,f,i) )
            IR_instruction = c("MOVE",
                c("TEMP",
                     c(i)
                 ),
                c("CALL",
                     c(f),
                     c("TEMP",
                          c(i)
                      )
                 )
            )
            print(IR_instruction)
            ret.append(IR_instruction)
IR(rete,True)

y = input
MOVE(TEMP(10),CONST(0))
MOVE(TEMP(11),CONST(0))
MOVE(TEMP(12),CONST(0))
MOVE(TEMP(13),CONST(0))
MOVE(TEMP(14),CONST(0))
MOVE(TEMP(15),CONST(0))
MOVE(TEMP(16),CONST(0))
MOVE(TEMP(17),CONST(0))
MOVE(TEMP(18),CONST(0))
MOVE(TEMP(19),CONST(0))
MOVE(TEMP(10),BINOP(ADD,TEMP(10),BINOP(MUL,CONST(0.556648971790991),TEMP(0))))
MOVE(TEMP(10),BINOP(ADD,TEMP(10),BINOP(MUL,CONST(-0.158044727952419),TEMP(0))))
MOVE(TEMP(10),BINOP(ADD,TEMP(10),BINOP(MUL,CONST(1.1529303537358564),TEMP(0))))
MOVE(TEMP(10),BINOP(ADD,TEMP(10),BINOP(MUL,CONST(-0.1419927818046811),TEMP(0))))
MOVE(TEMP(10),BINOP(ADD,TEMP(10),BINOP(MUL,CONST(-0.6396414495678946),TEMP(0))))
MOVE(TEMP(11),BINOP(ADD,TEMP(11),BINOP(MUL,CONST(1.333008772296133),TEMP(1))))
MOVE(TEMP(11),BINOP(ADD,TEMP(11),BINOP(MUL,CONST(0.7074249957921703),TEMP(1))))
MOVE(TEMP(11),BINOP(ADD,TEMP(11),BINOP(MUL,CONST(-0.5890229731507963),TEMP(1))))
MOVE(TEMP(11),BINOP(ADD,TEMP(11),BINOP(MUL,CONST(0.6939398194373148),TEMP(1))))
MOVE(TEMP(11),BINOP(ADD,TEMP(

# Assembling